# ASD Meta-Analysis

## <i> Figure 2d Alpha Diversity Log2 Fold Change Boxplots </i>
#### This notebook contains the steps to generate the file used to plot Figure 2d.

In [17]:
#Import dependencies
import os
import qiime2 as q2
import pandas as pd
import seaborn as sns
import seaborn as sb
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import scipy
from collections import Counter
from qiime2 import Visualization
from qiime2 import Metadata
from qiime2.plugins import feature_table
from qiime2 import Artifact
from functools import reduce
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from pandas import read_csv, Series, DataFrame

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

#### Load metadata files

In [18]:
amgut_bloom_filt = pd.read_csv("../../Amgut/Bloom_filt_data/American_gut_metadata.txt",sep='\t',index_col=0)
berding = pd.read_csv("../../Berding_2020/sample_metadata_rf.txt",sep='\t',index_col=0)
cao = pd.read_csv("../Metadata/meta-cao.txt",sep='\t',index_col=0)
chen = pd.read_csv("../Metadata/meta-chen.txt",sep='\t',index_col=0)
dan = pd.read_csv("../Metadata/meta-dan.txt",sep='\t',index_col=0)
david = pd.read_csv("../Metadata/meta-david.txt",sep='\t',index_col=0)
huang = pd.read_csv("../Metadata/meta-huang.txt",sep='\t',index_col=0)
fouquier = pd.read_csv("../Metadata/meta-fouquier.txt",sep='\t',index_col=0)
kang = pd.read_csv("../Metadata/meta-kang.txt",sep='\t',index_col=0)
kong = pd.read_csv("../Metadata/meta-kong.txt",sep='\t',index_col=0)
liu = pd.read_csv("../Metadata/meta-liu.txt",sep='\t',index_col=0)
son = pd.read_csv("../Metadata/meta-son.txt",sep='\t',index_col=0)
zou = pd.read_csv("../Metadata/meta-zou.txt",sep='\t',index_col=0)
zurita = pd.read_csv("../Metadata/meta-zurita.txt",sep='\t',index_col=0)

In [19]:
berding_stat = berding.drop(["Sex"], axis=1)
chen_stat = chen.drop(['Sex'], axis=1)
cao_stat = cao.drop(['Sex', 'Country', 'Sample_size'], axis=1)
dan_stat = dan.drop(['Sex', 'Country', 'Sample_size'], axis=1)
david_stat = david.drop(['Sex'], axis=1)
huang_stat = huang.drop(['Sex'], axis=1)
fouquier_stat = fouquier.drop(['Sex', 'Country', 'Sample_size'], axis=1)
kang_stat = kang.drop(['Sex'], axis=1)
kong_stat = kong.drop(['Sex'], axis=1)
liu_stat = liu.drop(['Sex', 'Country', 'Sample_size'], axis=1)
son_stat = son.drop(['Sex', 'Country', 'Sample_size'], axis=1)
zou_stat = zou.drop(['Sex', 'Country', 'Sample_size'], axis=1)
zurita_stat = zurita.drop(['Sex', 'Country', 'Sample_size'], axis=1)

#### Define user functions

In [37]:
# Functions to calculate the Log2 Fold Changes

def ttest(df,ttdf1,ttdf2):
    from scipy.stats import ttest_ind
    ttests=[]
    ttdf1 = ttdf1.transpose()
    ttdf2 = ttdf2.transpose()
    ttests = ttest_ind(ttdf1,ttdf2,equal_var=False,nan_policy='omit')
    ttests = ttests.pvalue.transpose()
    df['ttest_pvalue']=ttests
    
def fold(df, sub1, sub2, sub1name, sub2name):
    df['%s Mean' % (sub1name)]=sub1.mean(axis=1,skipna=True)
    df['%s Mean' % (sub2name)]=sub2.mean(axis=1,skipna=True)
    df['Overall_Mean'] = (df['%s Mean' % (sub1name)]+df['%s Mean' % (sub2name)])/2
    df['Fold Change(%s/%s)' % (sub1name,sub2name)]=df['%s Mean' % (sub1name)]/df['%s Mean' % (sub2name)]
    print("Fold Change Column Name = 'Fold Change(%s/%s)'" %(sub1name,sub2name))
    
def piscore(df,sub1name,sub2name):
    df['Log2(Fold Change)'] = np.log2(df['Fold Change(%s/%s)'% (sub1name,sub2name)])
    df['-Log(P-value)'] = -np.log10(df['ttest_pvalue'])
    df['%s/%s pi score' % (sub1name,sub2name)] = df['-Log(P-value)']*df['Log2(Fold Change)']
    df['Relevance_Score'] = 1/(-np.log10(abs(df['%s/%s pi score'% (sub1name,sub2name)])*df['Overall_Mean']))

def TwoCategoryMaster(dfmain,dfsub1,dfsub2,sub1name,sub2name):
    ttest(dfmain,dfsub1,dfsub2)
    fold(dfmain,dfsub1,dfsub2,sub1name,sub2name)
    piscore(dfmain,sub1name,sub2name)
    
def refactor_sig(x):
    if x < 0.05:
        return 'Significant'
    else:
        return 'p > 0.05'

In [74]:
def alpha_diversity(study):
    
    if study == 'amgut_bloom_filt':
        path = "../../Amgut/Bloom_filt_data"
        meta_file = amgut_bloom_filt
    if study == 'berding':
        path = "../../Berding_2020"
        meta_file = berding_stat
    if study == 'cao':
        path = "../../Cao_2021"
        meta_file = cao_stat
    if study == 'chen':
        path = "../../Chen_2020"
        meta_file = chen_stat
    if study == 'dan':
        path = "../../Dan_2020"
        meta_file = dan_stat
    if study == 'david':
        path = "../../David_2021"
        meta_file = david_stat
    if study == 'huang':
        path = "../../Huang_2021"
        meta_file = huang_stat
    if study == 'fouquier':
        path = "../../Fouquier_2021"
        meta_file = fouquier_stat
    if study == 'kang':
        path = "../../Kang_2017"
        meta_file = kang_stat
    if study == 'kong':
        path = "../../Kong_2019"
        meta_file = kong_stat
    if study == 'liu':
        path = "../../Liu_2019"
        meta_file = liu_stat
    if study == 'son':
        path = "../../Son_2015"
        meta_file = son_stat
    if study == 'zou':
        path = "../../Zou_2020"
        meta_file = zou_stat
    if study == 'zurita':
        path = "../../Zurita_2019"
        meta_file = zurita_stat

    alpha = Artifact.load('./%s/core-metrics-results-16S/faith_pd_vector.qza' % path).view(Series)
    shannon = Artifact.load('./%s/core-metrics-results-16S/shannon_vector.qza'% path).view(Series)
    evennness = Artifact.load('./%s/core-metrics-results-16S/evenness_vector.qza'% path).view(Series)
    obs_features = Artifact.load('./%s/core-metrics-results-16S/observed_features_vector.qza'% path).view(Series)
    metadata = meta_file.copy()
    meta_file = meta_file.loc[alpha.index].copy()
    meta_file['Subject_identifier'] = meta_file.index
    
    output = []
    recipient_table = []


    for subject_id, recipient_table in meta_file.groupby('Subject_identifier'):

        subject_id = recipient_table.Subject_identifier.iloc[0]

        faith = alpha[subject_id]
        shannon_value = shannon[subject_id]
        evennness_value = evennness[subject_id]
        obs_features_value = obs_features[subject_id]
        Study = recipient_table.Study.iloc[0]
        Status = recipient_table.Status.iloc[0] 

        output.append([subject_id, faith, shannon_value, evennness_value, obs_features_value, Study, Status])

    df = pd.DataFrame(columns=['subject_id', 'faith', 'shannon', 'evennness', 'obs_features', 'Study', 'Status'], data=output)
    
    df2 = df[['subject_id','faith','shannon','evennness','obs_features']]
    df2 = df2.set_index("subject_id")
        
    df3 = df2.T

    ASD_ids = list(metadata[metadata['Status']=='ASD'].index)
    Control_ids = list(metadata[metadata['Status']=='Control'].index)
    ASD_ids = list(set(ASD_ids)&set(df3.columns))
    Control_ids = list(set(Control_ids)&set(df3.columns))
    ASD = df3[ASD_ids]
    Control = df3[Control_ids]
    
    TwoCategoryMaster(dfmain=df3,dfsub1=ASD,
                  dfsub2=Control,sub1name='ASD',sub2name='Control')
    
    
    df4 = df3.rename(columns={"ttest_pvalue": "%s_pval" % study})
    df4 = df4.rename(columns={"Log2(Fold Change)": "%s_log2" % study})
    
    columns = ["%s_log2" % study, "%s_pval" % study] 
    df5 = df4[columns]
    
    return(df5)


In [75]:
studies = ["amgut_bloom_filt", "berding", "cao", "chen", "dan", "david", "huang", "fouquier", 
           "kang", "kong", "liu", "son", "zou", "zurita", ]

dataframes = {}

for s in studies:

    dataframes[f'{s}'] = alpha_diversity(s)

Fold Change Column Name = 'Fold Change(ASD/Control)'
Fold Change Column Name = 'Fold Change(ASD/Control)'
Fold Change Column Name = 'Fold Change(ASD/Control)'
Fold Change Column Name = 'Fold Change(ASD/Control)'
Fold Change Column Name = 'Fold Change(ASD/Control)'
Fold Change Column Name = 'Fold Change(ASD/Control)'
Fold Change Column Name = 'Fold Change(ASD/Control)'
Fold Change Column Name = 'Fold Change(ASD/Control)'
Fold Change Column Name = 'Fold Change(ASD/Control)'
Fold Change Column Name = 'Fold Change(ASD/Control)'
Fold Change Column Name = 'Fold Change(ASD/Control)'
Fold Change Column Name = 'Fold Change(ASD/Control)'
Fold Change Column Name = 'Fold Change(ASD/Control)'
Fold Change Column Name = 'Fold Change(ASD/Control)'


In [76]:
df = dataframes['amgut_bloom_filt'] # Initializes df

studies.remove('amgut_bloom_filt')

for s in studies:
    df=df.merge(dataframes[f'{s}'],left_index=True,right_index=True)
    

studies.append('amgut_bloom_filt')

In [84]:
pvalstudies=[]
for i in studies:
    a = i + '_pval'
    pvalstudies.append(a)
pvalstudies

['berding_pval',
 'cao_pval',
 'chen_pval',
 'dan_pval',
 'david_pval',
 'huang_pval',
 'fouquier_pval',
 'kang_pval',
 'kong_pval',
 'liu_pval',
 'son_pval',
 'zou_pval',
 'zurita_pval',
 'amgut_bloom_filt_pval']

In [78]:
value_varsstudies=[]
for i in studies:
    a = i + '_log2'
    value_varsstudies.append(a)
value_varsstudies

['berding_log2',
 'cao_log2',
 'chen_log2',
 'dan_log2',
 'david_log2',
 'huang_log2',
 'fouquier_log2',
 'kang_log2',
 'kong_log2',
 'liu_log2',
 'son_log2',
 'zou_log2',
 'zurita_log2',
 'amgut_bloom_filt_log2']

In [79]:
df["Alpha_diversity_metric"] = df.index
df

subject_id,amgut_bloom_filt_log2,amgut_bloom_filt_pval,berding_log2,berding_pval,cao_log2,cao_pval,chen_log2,chen_pval,dan_log2,dan_pval,...,kong_pval,liu_log2,liu_pval,son_log2,son_pval,zou_log2,zou_pval,zurita_log2,zurita_pval,Alpha_diversity_metric
faith,-0.101586,0.020058,0.114131,0.386990,-0.048171,0.299348,0.088742,0.056629,-0.002217,0.890900,...,0.038788,-0.033371,0.730147,0.044610,0.469773,-0.135085,0.009876,0.008727,0.925173,faith
shannon,-0.071795,0.027798,0.058274,0.396672,-0.107461,0.071880,-0.013353,0.669895,-0.001303,0.940831,...,0.258256,-0.100617,0.024650,0.009103,0.838313,0.157297,0.003567,0.013209,0.830063,shannon
evennness,-0.042908,0.096611,0.039750,0.429551,-0.106268,0.047143,-0.045129,0.057592,-0.003565,0.813658,...,0.525306,-0.081143,0.005329,0.007586,0.839773,0.177178,0.000215,0.007182,0.863716,evennness
obs_features,-0.138170,0.012621,0.132223,0.328646,-0.010269,0.858485,0.143443,0.020768,0.011538,0.586288,...,0.034770,-0.054900,0.633110,0.018091,0.777771,-0.083903,0.219739,0.033159,0.794712,obs_features


In [80]:
FC_melt = pd.melt(frame=df, id_vars='Alpha_diversity_metric',value_vars=value_varsstudies)
FC_melt.rename(columns={'variable':'Cohort','value':'Log2(ASD/Control)'},inplace=True)

In [85]:
pval_melt = pd.melt(frame=df, id_vars='Alpha_diversity_metric',value_vars=pvalstudies)
pval_melt.rename(columns={'variable':'Cohort','value':'p_value'},inplace=True)

In [91]:
df_m=pval_melt.merge(FC_melt,left_index=True,right_index=True)
df_m.rename(columns={'Alpha_diversity_metric_x':'Phyla','Cohort_x':'Cohort'},inplace=True)
df_m['Log2(ASD/Control)']=df_m['Log2(ASD/Control)'].astype('float')

In [92]:
df_m['Significant'] = df_m['p_value'].apply(refactor_sig)

In [93]:
df_m['subject_id_x']=df_m['subject_id_x'].str.replace('_pval','')
del df_m["subject_id_y"]
del df_m["Alpha_diversity_metric_y"]
df_m.rename(columns={'Phyla':'Alpha_diversity_metric'},inplace=True)
df_m.rename(columns={'subject_id_x':'Study'},inplace=True)
df_m = df_m.set_index("Alpha_diversity_metric")

In [94]:
df_m.to_csv("per_study_alpha_diversity_log2fc.txt", sep='\t')